# Library & Data Loading

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install konlpy &> /dev/null
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git &> /dev/null
!bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh &> /dev/null

In [3]:
from collections import defaultdict
from tqdm import tqdm
from konlpy.tag import Mecab
from konlpy.corpus import kolaw
import pandas as pd
import numpy as np
import pickle

In [6]:
df = pd.read_csv("/content/drive/MyDrive/dissertation/baemin_mecab_v02.csv")
# df = pd.read_csv("/content/drive/MyDrive/dissertation/yogiyo_mecab_v02.csv")
# df = pd.read_csv("/content/drive/MyDrive/dissertation/coupang_mecab_v02.csv")
df.head()

,userName,content,score,reviewCreatedVersion,at,morpheme
0,크레아틴,월간쿠폰 최소주문금액 왜 늘렸습니까 솔직히 답변 바랍니다,1,11.24.0,2022-06-13 13:32,"['할인쿠폰', '주문', '음식값', '대응']"
1,쌀떡찹,야금야금 쿠폰 사용 최소 금액 올리는거 역겨움의 극치임 제발 업보 쌓은대로 돌려받...,1,11.24.0,2022-06-13 12:01,"['할인쿠폰', '사용', '음식값', '극치', '업보', '대응', '매크로',..."
2,홍짱,이름만 바꾼 똑같은 식당좀 안보이게해주면 안되나요,1,11.24.0,2022-06-13 8:18,"['이름', '같다', '매장']"
3,bz z,돈에 미친 대기업의 표본,1,11.18.1,2022-06-13 8:10,"['돈', '기업', '표본']"
4,yujoo,비마트는 문앞 놓고가는 선택지도 필요한듯해요 그리고 왜 배민과 달리 비마트는 평소 ...,1,11.24.0,2022-06-13 2:57,"['비마트', '현관문', '선택지', '필요하다', '비마트', '요청', '사항..."


# N-Gram

In [8]:
mecab = Mecab()     # mecab 말고 okt, komoran 취사 선택 가능

def get_ngram_counter(docs, min_count=10, n_range=(2,4)):       # min_count: 최소 빈도수 / n_range: n-gram의 n 설정하기(1부터 3)

    def to_ngrams(words, n):
        ngrams = []
        for b in range(0, len(words) - n + 1):
            ngrams.append(tuple(words[b:b+n]))
        return ngrams

    n_begin, n_end = n_range
    ngram_counter = defaultdict(int)
    for doc in tqdm(docs):
        words = mecab.pos(doc, join=True)       # mecab 말고 okt, komoran 취사 선택 가능
        for n in range(n_begin, n_end + 1):
            for ngram in to_ngrams(words, n):
                ngram_counter[ngram] += 1

    ngram_counter = {
        ngram:count for ngram, count in ngram_counter.items()
        if count >= min_count
    }

    return ngram_counter

In [10]:
## ngram 계산하기

# df['content'] 부분에 data 본문부분 삽입, 전처리 잘 안돼있으면 오류
# 2gram, 3gram 보려면 n_range=(2,3) / 2gram, 3gram, 4gram 보려면 n_range=(2,4)

ngram_counter = get_ngram_counter(df['content'], min_count=10, n_range=(2,4))

100%|██████████| 68402/68402 [00:31<00:00, 2173.71it/s]


In [25]:
## ngram 결과 보기

# keyword에 보고자 하는 단어/품사 삽입
# mecab 기준 NNG(혹은 NNP): 명사, VA: 형용사 / okt나 komoran은 다름

for ngram, count in sorted(ngram_counter.items(), key=lambda x:-x[1]):
    keyword = '지역/NNG'
    if ngram[0] == keyword:
        print(ngram, count)
    elif ngram[-1] == keyword: 
        print(ngram, count) 

# 검색기록
# '많/VA','같/VA','가능/NNG','필요/NNG','기능/NNG','덕분/NNG','사항/NNG','배달/NNG','주문/NNG','금액/NNG','시간/NNG',
# '서비스/NNG','삭제/NNG','검색/NNG','선택/NNG','접수/NNG','방법/NNG','지역/NNG','확인/NNG','혜택/NNG',

('지역/NNG', '에/JKB') 177
('는/ETM', '지역/NNG') 170
('지역/NNG', '이/VCP') 140
('배달/NNG', '지역/NNG') 137
('가능/NNG', '지역/NNG') 113
('다른/MM', '지역/NNG') 113
('지역/NNG', '에서/JKB') 100
('배달/NNG', '가능/NNG', '지역/NNG') 91
('어느/MM', '지역/NNG') 88
('지역/NNG', '이/JKS') 86
('지역/NNG', '을/JKO') 84
('불가/NNG', '지역/NNG') 84
('타/MM', '지역/NNG') 78
('배달/NNG', '불가/NNG', '지역/NNG') 76
('지역/NNG', '별/XSN') 70
('지역/NNG', '은/JX') 63
('지역/NNG', '이/JKC') 58
('지역/NNG', '이/VCP', '라고/EC') 56
('지역/NNG', '도/JX') 54
('지역/NNG', '으로/JKB') 48
('그/MM', '지역/NNG') 47
('지역/NNG', '별/XSN', '로/JKB') 46
('지역/NNG', '이/JKC', '아니/VCN') 44
('지역/NNG', '을/JKO', '가/VV') 44
('되/VV', '는/ETM', '지역/NNG') 43
('안/MAG', '되/VV', '는/ETM', '지역/NNG') 40
('지역/NNG', '설정/NNG') 35
('우리/NP', '지역/NNG') 34
('지역/NNG', '에/JKB', '가/VV') 34
('지역/NNG', '에서/JKB', '도/JX') 34
('지역/NNG', '마다/JX') 33
('지역/NNG', '의/JKG') 32
('지역/NNG', '배달/NNG') 30
('고/EC', '지역/NNG') 30
('한/XSA+ETM', '지역/NNG') 30
('모르/VV', '는/ETM', '지역/NNG') 29
('지역/NNG', '만/JX') 27
('지역/NNG', '인데/VCP+EC') 26
(